In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
pwd

'f:\\projects\\project-4 chicken disease predictor\\cd predict\\chicken-project'

In [4]:
from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen = True)
class DataIngestionConfig:
    root_dir:Path
    source_URL_1: str
    local_data_file_1: Path
    source_URL_2: str
    local_data_file_2: Path
    unzip_dir: Path

In [5]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath =  CONFIG_FILE_PATH,
        params_filepath =  PARAMS_FILE_PATH
        ):
                     
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
        
        
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL_1 = config.source_URL_1,
            local_data_file_1 = config.local_data_file_1,
            source_URL_2 = config.source_URL_2,
            local_data_file_2 = config.local_data_file_2,
            unzip_dir = config.unzip_dir
        )
        
        return data_ingestion_config

In [7]:
import os
import urllib.request as request
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [8]:
class DataIngestion:
    def __init__(self,config: DataIngestionConfig):
        self.config = config
        
    def download_file_1(self):
        if not os.path.exists(self.config.local_data_file_1):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL_1,
                filename = self.config.local_data_file_1
            )
            
            logger.info(f"{filename} download! with following info: \n {headers}")
            
        else:
            logger.info(f"file already exists of size: {get_size(Path(self.config.local_data_file_1))}")
            
            
    def download_file_2(self):
        if not os.path.exists(self.config.local_data_file_2):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL_2,
                filename = self.config.local_data_file_2
            )
                
            logger.info(f"{filename} download! with following info: \n {headers}")
        else:
            logger.info(f"file already exists of size: {get_size(Path(self.config.local_data_file_2))}")

            
            
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        function
        """
        
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok = True)
        with zipfile.ZipFile(self.config.local_data_file_1,"r") as zip_ref:
            zip_ref.extractall(unzip_path)
            logger.info(f"extracted to {unzip_path}")
            
        with zipfile.ZipFile(self.config.local_data_file_2,"r") as zip_ref:
            zip_ref.extractall(unzip_path)
            logger.info(f"extracted to {unzip_path}")
            
    

In [9]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file_1()
    data_ingestion.download_file_2()
    
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e
    

[2025-12-02 15:23:39,859: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-12-02 15:23:39,898: INFO: common: yaml file: params.yaml loaded successfully]
[2025-12-02 15:23:39,902: INFO: common: created directory at : artifacts]
[2025-12-02 15:23:39,906: INFO: common: created directory at : artifacts/data_ingestion]
[2025-12-02 15:23:39,910: INFO: 1882738956: file already exists of size: ~ 23628 KB]
[2025-12-02 15:23:39,913: INFO: 1882738956: file already exists of size: ~ 25400 KB]
[2025-12-02 15:23:40,390: INFO: 1882738956: extracted to artifacts/data_ingestion]
[2025-12-02 15:23:40,861: INFO: 1882738956: extracted to artifacts/data_ingestion]
